In [1]:
%%time
import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.model.DecisionTreeModel
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.60:4040
SparkContext available as 'sc' (version = 3.0.0-preview2, master = local[*], app id = local-1592146840056)
SparkSession available as 'spark'


Time: 36.49453783035278 seconds.



import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.model.DecisionTreeModel
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.classification.{LogisticRegressionModel, LogisticRegressionWithLBFGS}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.regression.LabeledPoint


In [27]:
val data = MLUtils.loadLibSVMFile(sc, "a1a.txt")

data: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[86] at map at MLUtils.scala:86


In [29]:
val splits = data.randomSplit(Array(0.7, 0.3))
val (trainingData, testData) = (splits(0), splits(1))

splits: Array[org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint]] = Array(MapPartitionsRDD[89] at randomSplit at <console>:48, MapPartitionsRDD[90] at randomSplit at <console>:48)
trainingData: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[89] at randomSplit at <console>:48
testData: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[90] at randomSplit at <console>:48


In [30]:
trainingData

res10: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[89] at randomSplit at <console>:48


In [31]:
testData

res11: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[90] at randomSplit at <console>:48


In [32]:
val numClasses = 2
val categoricalFeaturesInfo = Map[Int, Int]()
val impurity = "gini"
val maxDepth = 5
val maxBins = 32

numClasses: Int = 2
categoricalFeaturesInfo: scala.collection.immutable.Map[Int,Int] = Map()
impurity: String = gini
maxDepth: Int = 5
maxBins: Int = 32


In [33]:
val model = DecisionTree.trainClassifier(trainingData, numClasses, categoricalFeaturesInfo,
  impurity, maxDepth, maxBins)

model: org.apache.spark.mllib.tree.model.DecisionTreeModel = DecisionTreeModel classifier of depth 1 with 3 nodes


In [34]:
val labelAndPreds = testData.map { point =>
  val prediction = model.predict(point.features)
  (point.label, prediction)
}

labelAndPreds: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[103] at map at <console>:45


In [35]:
val testErr = labelAndPreds.filter(r => r._1 != r._2).count().toDouble / testData.count()
println(s"Test Error = $testErr")
println(s"Learned classification tree model:\n ${model.toDebugString}")

Test Error = 0.03125
Learned classification tree model:
 DecisionTreeModel classifier of depth 1 with 3 nodes
  If (feature 434 <= 79.5)
   Predict: 0.0
  Else (feature 434 > 79.5)
   Predict: 1.0



testErr: Double = 0.03125


In [36]:
// Save and load model
model.save(sc, "target/tmp/myDecisionTreeClassificationModel")
val sameModel = DecisionTreeModel.load(sc, "target/tmp/myDecisionTreeClassificationModel")

sameModel: org.apache.spark.mllib.tree.model.DecisionTreeModel = DecisionTreeModel classifier of depth 1 with 3 nodes
